In [1]:
import pandas as pd
import os

# Data Recruitment

## Ingesting Data

In [2]:
req_path = 'data/data_reqruitment/data_requirements.csv'

df_recruitments_orignal = pd.read_csv(req_path)

df_recruitments = df_recruitments_orignal.copy()
df_recruitments.drop(columns='Unnamed: 0', inplace=True)
df_recruitments.head()

,company,company_rating,location,job_title,job_description,salary_estimate,company_size,company_type,company_sector,company_industry,company_founded,company_revenue,dates
0,PCS Global Tech\n4.7,4.7,"Riverside, CA",Data Engineer | PAID BOOTCAMP,Responsibilities\n· Analyze and organize raw d...,"$70,000 /yr (est.)",501 to 1000 Employees,Company - Private,Information Technology,Information Technology Support Services,NaN,Unknown / Non-Applicable,2024-06-12 00:00:00-10:00
1,Futuretech Consultants LLC,NaN,"Newton, MS",Snowflake Data Engineer,My name is Dileep and I am a recruiter at Futu...,$42.50 /hr (est.),NaN,NaN,NaN,NaN,NaN,NaN,2024-06-12 00:00:00+07:00
2,Clairvoyant\n4.4,4.4,Remote,Data Engineer (MDM),Required Skills:\nMust have 5-8+ Years of expe...,$67.50 /hr (est.),51 to 200 Employees,Company - Private,Pharmaceutical & Biotechnology,Biotech & Pharmaceuticals,NaN,Unknown / Non-Applicable,2024-06-12 00:00:00-10:00
3,Apple\n4.2,4.2,"Cupertino, CA",Data Engineer,"Summary\nPosted: Dec 22, 2021\nWeekly Hours: 4...",NaN,10000+ Employees,Company - Public,Information Technology,Computer Hardware Development,1976.0,$10+ billion (USD),2024-06-12 00:00:00-05:00
4,Skytech Consultancy Services\n5.0,5.0,"Baltimore, MD",Data Engineer,Description of Work:\nTechnical experience in ...,$65.00 /hr (est.),1 to 50 Employees,Company - Public,NaN,NaN,NaN,Unknown / Non-Applicable,2024-06-12 00:00:00-04:00


## Cleaning & Transforming Data

In [3]:
df_recruitments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1555 entries, 0 to 1554
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   company           1551 non-null   object 
 1   company_rating    1358 non-null   float64
 2   location          1554 non-null   object 
 3   job_title         1554 non-null   object 
 4   job_description   1554 non-null   object 
 5   salary_estimate   1277 non-null   object 
 6   company_size      1442 non-null   object 
 7   company_type      1442 non-null   object 
 8   company_sector    1260 non-null   object 
 9   company_industry  1260 non-null   object 
 10  company_founded   1131 non-null   float64
 11  company_revenue   1442 non-null   object 
 12  dates             1555 non-null   object 
dtypes: float64(2), object(11)
memory usage: 158.1+ KB


In [4]:
# Ada beberapa error dalam kolom 'company' yang memasukan 'company rating'

# Regex untuk mendapatkan tiap elemen selain '\n'
clenased_company = df_recruitments['company'].str.findall(r'[a-zA-z0-9 ]+')

# Mengambil nama 'company' dengan asumsi index 0
def getFirstItem(item):
    try:
        fullName = str(item[0])
        return fullName
    except:
        return None

# apply dan menggantikannya kepada kolom 'company'
df_recruitments['company'] = clenased_company.apply(getFirstItem).astype('string')

In [5]:
# change categorical columns into string

df_recruitments['location'] = df_recruitments['location'].astype('string')
df_recruitments['job_title'] = df_recruitments['job_title'].astype('string') 
df_recruitments['job_description'] = df_recruitments['job_description'].astype('string')
df_recruitments['company_size'] = df_recruitments['company_size'].astype('string')
df_recruitments['company_type'] = df_recruitments['company_type'].astype('string')
df_recruitments['company_sector'] = df_recruitments['company_sector'].astype('string')
df_recruitments['company_revenue'] = df_recruitments['company_revenue'].astype('string')

In [6]:
# creating a 'numeric_salary' column for analysis
df_recruitments['numeric_salary'] = df_recruitments['salary_estimate'].str.replace(',', '').str.findall('[0-9.]+').apply(getFirstItem).astype(float)

In [7]:
# creating a 'salary_type' column to get categorical values
slrType = df_recruitments['salary_estimate'].str.findall('/[a-zA-Z]+')

def salaryType(salType):
    salary = getFirstItem(salType)

    if salary == '/yr':
        return 'yearly'
    elif salary == '/hr':
        return 'hourly'
    elif salary == '/mo':
        return 'monthly'
    else:
        return None

df_recruitments['salary_type'] = slrType.apply(salaryType).astype('string')

In [8]:
# creating a 'monthly_salary' column for easier salary comparison in analysis

def getMonthlySalary(x):
    try:
        if x['salary_type'] == 'yearly':
            salary = x['numeric_salary']/12
            salary = round(salary, 2)
            return salary
        elif x['salary_type'] == 'hourly':
            salary = x['numeric_salary'] * 173.2
            salary = round(salary, 2)
            return salary
        else:
            return None
    except:
        return None


df_recruitments['monthly_salary'] = df_recruitments.apply(getMonthlySalary, axis=1)

In [9]:
# pastiin dulu utc nya gmn
# df_recruitments['dates'] = pd.to_datetime(df_recruitments['dates'], utc=True)

In [10]:
df_recruitments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1555 entries, 0 to 1554
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   company           1551 non-null   string 
 1   company_rating    1358 non-null   float64
 2   location          1554 non-null   string 
 3   job_title         1554 non-null   string 
 4   job_description   1554 non-null   string 
 5   salary_estimate   1277 non-null   object 
 6   company_size      1442 non-null   string 
 7   company_type      1442 non-null   string 
 8   company_sector    1260 non-null   string 
 9   company_industry  1260 non-null   object 
 10  company_founded   1131 non-null   float64
 11  company_revenue   1442 non-null   string 
 12  dates             1555 non-null   object 
 13  numeric_salary    1277 non-null   float64
 14  salary_type       1277 non-null   string 
 15  monthly_salary    1269 non-null   float64
dtypes: float64(4), object(3), string(9)
memory

## Data Demography

Dari data yang kita sudah bersihkan diatas, kita menemukan beberapa kolom categorical yang kita bisa analisis dengan grouping yaitu `company_type`, `company_sector`, `company_industry`. Lalu kita juga telah membersihkan kolom `company_rating` dan membentuk `monthly_salary` untuk values yang kita bisa lihat dari analisis.

In [11]:
df_recruitments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1555 entries, 0 to 1554
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   company           1551 non-null   string 
 1   company_rating    1358 non-null   float64
 2   location          1554 non-null   string 
 3   job_title         1554 non-null   string 
 4   job_description   1554 non-null   string 
 5   salary_estimate   1277 non-null   object 
 6   company_size      1442 non-null   string 
 7   company_type      1442 non-null   string 
 8   company_sector    1260 non-null   string 
 9   company_industry  1260 non-null   object 
 10  company_founded   1131 non-null   float64
 11  company_revenue   1442 non-null   string 
 12  dates             1555 non-null   object 
 13  numeric_salary    1277 non-null   float64
 14  salary_type       1277 non-null   string 
 15  monthly_salary    1269 non-null   float64
dtypes: float64(4), object(3), string(9)
memory

In [12]:
df_recruitments.columns

Index(['company', 'company_rating', 'location', 'job_title', 'job_description',
       'salary_estimate', 'company_size', 'company_type', 'company_sector',
       'company_industry', 'company_founded', 'company_revenue', 'dates',
       'numeric_salary', 'salary_type', 'monthly_salary'],
      dtype='object')

Mencari unique values untuk masing-masing categorical variables

In [13]:
df_recruitments['company_type'].unique()

<StringArray>
[             'Company - Private',                             <NA>,
               'Company - Public',                       'Hospital',
                       'Contract', 'Subsidiary or Business Segment',
         'Nonprofit Organization',        'Private Practice / Firm',
                     'Government',                  'Self-employed',
           'College / University',                        'Unknown',
       'School / School District']
Length: 13, dtype: string

In [14]:
df_recruitments['company_sector'].unique()

<StringArray>
[                     'Information Technology',
                                          <NA>,
              'Pharmaceutical & Biotechnology',
                     'Management & Consulting',
                                  'Healthcare',
                  'Human Resources & Staffing',
                               'Manufacturing',
                          'Financial Services',
               'Hotels & Travel Accommodation',
                  'Transportation & Logistics',
                          'Retail & Wholesale',
            'Arts, Entertainment & Recreation',
          'Government & Public Administration',
                         'Aerospace & Defense',
                  'Personal Consumer Services',
                       'Media & Communication',
                  'Energy, Mining & Utilities',
                                   'Insurance',
 'Construction, Repair & Maintenance Services',
                                 'Agriculture',
                          

In [15]:
print(len(df_recruitments['company_industry'].unique()))
# memiliki 84 columns

df_recruitments['company_industry'].unique()

84


array(['Information Technology Support Services', nan,
       'Biotech & Pharmaceuticals', 'Computer Hardware Development',
       'Research & Development', 'Health Care Services & Hospitals',
       'Internet & Web Services', 'HR Consulting',
       'Consumer Product Manufacturing',
       'Enterprise Software & Network Solutions',
       'Investment & Asset Management', 'Accounting & Tax',
       'Business Consulting', 'Financial Transaction Processing',
       'Staffing & Subcontracting', 'Banking & Lending',
       'Travel Agencies', 'Airlines, Airports & Air Transportation',
       'Automotive Parts & Accessories Stores', 'Sports & Recreation',
       'National Agencies', 'Aerospace & Defense', 'Software Development',
       'Beauty & Wellness', 'Publishing', 'Food & Beverage Manufacturing',
       'Energy & Utilities', 'Advertising & Public Relations',
       'Insurance Carriers', 'Wholesale',
       'Department, Clothing & Shoe Stores', 'Machinery Manufacturing',
       'Archite

In [16]:
df_recruitments.describe()

,company_rating,company_founded,numeric_salary,monthly_salary
count,1358.000000,1131.000000,1277.000000,1269.000000
mean,3.925626,1975.934571,98197.904933,9807.472459
std,0.535826,51.461792,50571.949049,2798.779490
min,1.000000,1636.000000,22.500000,3897.000000
25%,3.600000,1965.000000,84414.000000,7883.330000
50%,3.900000,1997.000000,102793.000000,9350.500000
75%,4.200000,2009.000000,125000.000000,11250.000000
max,5.000000,2022.000000,341000.000000,28416.670000


### Company Rating 

In [17]:
df_recruitments.groupby('company_type')['company_rating'].agg(['mean', 'min', 'max', 'count']).sort_values(by='mean', ascending=False)

,mean,min,max,count
company_type,,,,
College / University,4.193333,3.9,4.5,15
Private Practice / Firm,4.030000,3.4,4.5,10
Unknown,4.000000,3.1,4.6,8
Company - Private,3.988143,2.0,5.0,700
Company - Public,3.891813,1.3,5.0,513
Nonprofit Organization,3.729545,2.2,4.6,44
Subsidiary or Business Segment,3.721875,2.9,4.7,32
School / School District,3.700000,3.7,3.7,1
Contract,3.666667,1.0,5.0,12


In [18]:
df_recruitments.groupby('company_sector')['company_rating'].agg(['mean', 'min', 'max', 'count']).sort_values(by='mean', ascending=False)

,mean,min,max,count
company_sector,,,,
Human Resources & Staffing,4.192857,3.4,5.0,28
Hotels & Travel Accommodation,4.075000,3.3,4.5,4
Information Technology,4.050000,1.3,5.0,500
"Arts, Entertainment & Recreation",4.046154,3.1,5.0,13
Aerospace & Defense,3.997674,3.5,4.8,43
Education,3.921429,3.0,4.5,28
Real Estate,3.910000,3.5,4.7,10
Pharmaceutical & Biotechnology,3.850000,2.8,4.6,28
Management & Consulting,3.850000,2.4,4.8,72


In [19]:
df_recruitments.groupby('company_industry')['company_rating'].agg(['mean', 'min', 'max', 'count']).sort_values(by='mean', ascending=False)

,mean,min,max,count
company_industry,,,,
Consumer Electronics & Appliances Stores,4.500000,4.5,4.5,1
Automotive Parts & Accessories Stores,4.333333,3.9,4.8,3
Gambling,4.300000,3.9,5.0,3
Staffing & Subcontracting,4.228571,3.6,4.5,7
Publishing,4.200000,4.1,4.3,2
...,...,...,...,...
Drug & Health Stores,3.100000,3.1,3.1,1
Religious Institutions,3.100000,3.1,3.1,1
Building & Personnel Services,3.100000,2.4,3.8,2


### Monthy Salary

In [20]:
df_recruitments.groupby('company_type')['monthly_salary'].agg(['mean', 'min', 'max', 'count']).sort_values(by='mean', ascending=False)

,mean,min,max,count
company_type,,,,
Self-employed,11875.000000,10625.00,12500.00,3
Company - Public,10249.486516,4239.58,28416.67,465
Company - Private,9668.737496,3897.00,27712.00,595
Unknown,9348.728333,5303.92,13261.17,12
Government,9032.188182,6422.58,12142.00,11
Private Practice / Firm,9023.546667,5657.67,16666.67,9
Subsidiary or Business Segment,8881.402069,5238.42,14202.40,29
College / University,8607.819091,6144.17,11620.75,11
Nonprofit Organization,8315.824324,5385.33,12050.00,37


In [21]:
df_recruitments.groupby('company_sector')['monthly_salary'].agg(['mean', 'min', 'max', 'count']).sort_values(by='mean', ascending=False)

,mean,min,max,count
company_sector,,,,
Telecommunications,11641.726000,8071.33,16550.00,10
Hotels & Travel Accommodation,11239.585000,8500.00,12291.67,4
Information Technology,10484.207359,3897.00,28416.67,390
Human Resources & Staffing,10176.908276,4760.42,16666.67,29
Management & Consulting,10114.562985,4330.00,20208.33,67
Media & Communication,9926.274815,6188.17,14787.50,27
Pharmaceutical & Biotechnology,9913.469167,5306.67,17500.00,24
Aerospace & Defense,9428.791875,6183.50,15833.33,32
Restaurants & Food Service,9425.492727,4854.42,12788.17,11


In [22]:
df_recruitments.groupby('company_type')['company_rating'].agg(['mean', 'min', 'max', 'count']).sort_values(by='mean', ascending=False)

,mean,min,max,count
company_type,,,,
College / University,4.193333,3.9,4.5,15
Private Practice / Firm,4.030000,3.4,4.5,10
Unknown,4.000000,3.1,4.6,8
Company - Private,3.988143,2.0,5.0,700
Company - Public,3.891813,1.3,5.0,513
Nonprofit Organization,3.729545,2.2,4.6,44
Subsidiary or Business Segment,3.721875,2.9,4.7,32
School / School District,3.700000,3.7,3.7,1
Contract,3.666667,1.0,5.0,12


In [23]:
df_recruitments.groupby('company_industry')['monthly_salary'].agg(['mean', 'min', 'max', 'count']).sort_values(by='mean', ascending=False)

,mean,min,max,count
company_industry,,,,
Video Game Publishing,14787.500000,14787.50,14787.50,1
Security & Protective,14784.982000,9133.25,20208.33,5
Travel Agencies,12291.670000,12291.67,12291.67,2
"Cable, Internet & Telephone Providers",12067.543333,8363.17,16550.00,6
Grantmaking & Charitable Foundations,11910.415000,11770.83,12050.00,2
...,...,...,...,...
Pet & Pet Supplies Stores,6070.080000,4840.58,7299.58,2
Medical Testing & Clinical Laboratories,5303.920000,5303.92,5303.92,1
Consumer Electronics & Appliances Stores,NaN,NaN,NaN,0


# Data Products

## Extracting Data from Source Folder

In [24]:
# Creating a list with file names in the data_products folder
destination_path = 'data/data_products'
all_files = [file for file in os.listdir(destination_path) if file.endswith('.csv')]

# Create a list of dataframes for each files in the data_products folder
dataframes = []
for i in all_files:
    file = os.path.join(destination_path, i)
    df = pd.read_csv(file)
    dataframes.append(df)

# Concate all dataframes in the 'dataframes' lists
df_products_original = pd.concat(dataframes, ignore_index=True)

In [25]:
# Creating a copy to preserve original dataframes
df_products = df_products_original.copy()

df_products.head()

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,Unnamed: 0
0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/31UISB90sY...,https://www.amazon.in/Lloyd-Inverter-Convertib...,4.2,"2,255","₹32,999","₹58,990",NaN
1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.2,"2,948","₹46,490","₹75,990",NaN
2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Inverter-Convertible-...,4.2,"1,206","₹34,490","₹61,990",NaN
3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.0,69,"₹37,990","₹68,990",NaN
4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41lrtqXPiW...,https://www.amazon.in/Carrier-Inverter-Split-C...,4.1,630,"₹34,490","₹67,790",NaN


## Cleaning and Transforming Data

In [26]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103170 entries, 0 to 1103169
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   name            1103170 non-null  object 
 1   main_category   1103170 non-null  object 
 2   sub_category    1103170 non-null  object 
 3   image           1103170 non-null  object 
 4   link            1103170 non-null  object 
 5   ratings         751582 non-null   object 
 6   no_of_ratings   751582 non-null   object 
 7   discount_price  980844 non-null   object 
 8   actual_price    1067544 non-null  object 
 9   Unnamed: 0      551585 non-null   float64
dtypes: float64(1), object(9)
memory usage: 84.2+ MB


Penyelidikan terhadap data. Untuk sejauh ini kita bisa lihat bahwa terdapat beberapa null value dalam beberapa kolom itu. Kita bisa melihat bahwa untuk kolom ```name, main_category, sub_category, image, link``` seharusnya memiliki tipe data string. Selain itu untuk kolom ```ratings, no_of_ratings, discount_price, actual_price``` yang seharusnya memiliki tipe data interger atau float. Terakhir, terdapat kolom ```Unnamed: 0``` yang perlu kita selidiki.

In [27]:
# Mengubah kolom name, main_category, sub_category, image, dan link menjadi string

df_products['name'] = df_products['name'].astype('string')
df_products['main_category'] = df_products['main_category'].astype('string')
df_products['sub_category'] = df_products['sub_category'].astype('string')
df_products['image'] = df_products['image'].astype('string')
df_products['link'] = df_products['link'].astype('string')

In [28]:
# Untuk melihat bentuk data tpe masing-masing kolom dan jumlah non-null values

df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103170 entries, 0 to 1103169
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   name            1103170 non-null  string 
 1   main_category   1103170 non-null  string 
 2   sub_category    1103170 non-null  string 
 3   image           1103170 non-null  string 
 4   link            1103170 non-null  string 
 5   ratings         751582 non-null   object 
 6   no_of_ratings   751582 non-null   object 
 7   discount_price  980844 non-null   object 
 8   actual_price    1067544 non-null  object 
 9   Unnamed: 0      551585 non-null   float64
dtypes: float64(1), object(4), string(5)
memory usage: 84.2+ MB


Lalu kita akan lanjut untuk melihat bagaimana cara menghandle numeric data type. Setelah diselidiki ditemukan beberapa non-numeric values dimasing-masing kolom. Oleh karena itu, melainkan menggunakan ```astype()``` method, kita menggunakan ```apply()``` method untuk mengubah values dari string (atau tipe data yang lain) menjadi ```int/float```. Untuk non-numeric kita akan ubah valuenya menjadi ```0``` tapi juga akan tetap melihat kasus unik masing-masing kolom.

In [29]:
# Membuat fungsi untuk mengubah menjadi interger atau float

def getFloat(number):
    try:
        number = float(number)
        return number
    except:
        return None   

In [30]:
df_products['ratings'].unique()

array(['4.2', '4.0', '4.1', '4.3', '3.9', '3.8', '3.5', nan, '4.6', '3.3',
       '3.4', '3.7', '2.9', '5.0', '4.4', '3.6', '2.7', '4.5', '3.0',
       '3.1', '3.2', '4.8', '4.7', '2.5', '1.0', '2.6', '2.8', '2.3',
       '1.7', 'Get', '1.8', '2.4', '2.0', '1.5', '4.9', '1.9', '2.2',
       '1.2', '2.1', '1.4', '1.6', '1.3', 'FREE', '₹2.99', '1.1', '₹70',
       4.5, 3.3, 3.7, 4.2, 4.3, 4.1, 3.8, 4.4, 4.0, 3.5, 3.9, '₹99', 3.4,
       3.6, 3.2, 5.0, 2.6, 4.7, 3.0, 3.1, 4.6, 1.0, 4.9, 2.9, 2.7, 2.2,
       2.5, 4.8, 2.0, 1.7, 1.9, 2.8, 2.4, 1.6, 2.3, 1.8, 1.4, 1.3, 2.1,
       1.5, '₹100', '₹68.99', '₹65'], dtype=object)

In [31]:
df_products['ratings'] = df_products['ratings'].apply(getFloat)

In [32]:
# Menghilangkan ',' agar dapat diubah menjadi interger
df_products['no_of_ratings'] = df_products['no_of_ratings'].str.replace(',', '')

# Untuk bantu check non number valuesnya seperti apa. karena kolom 'no_of_ratings' memiliki unique values sangat banyak
def getNonNumber(number):
    try:
        number = float(number)
        return 0
    except:
        return number

test_norating = df_products['no_of_ratings'].apply(getNonNumber)
test_norating.unique()

# Karena mengetahui bahwa angka non-numeric values merupaka string error yang tidak dimiliki kolom lain, kita dapat mengubahnya menjadi nilai 0

array([0, 'Only 2 left in stock.', 'Only 1 left in stock.',
       'FREE Delivery by Amazon', 'Usually dispatched in 3 to 4 weeks.',
       'Usually dispatched in 5 to 6 days.',
       'Usually dispatched in 4 to 5 days.',
       'Usually dispatched in 6 to 7 days.',
       'Usually dispatched in 7 to 8 days.',
       'Usually dispatched in 11 to 12 days.',
       'Usually dispatched in 4 to 5 weeks.', 'Only 4 left in stock.',
       'Only 3 left in stock.', 'Usually dispatched in 1 to 2 months.',
       'Only 5 left in stock.',
       'This item will be released on August 14 2023.',
       'Usually dispatched in 3 to 5 days.',
       'Usually dispatched in 2 to 3 days.',
       'Usually dispatched in 9 to 10 days.',
       'Usually dispatched in 2 to 3 weeks.',
       'Usually dispatched in 8 to 9 days.'], dtype=object)

In [33]:
df_products['no_of_ratings'] = df_products['no_of_ratings'].apply(getFloat)
df_products['no_of_ratings'] = df_products['no_of_ratings'].astype('Int64')

In [34]:
# Bersihkan string agar dapat diubah
df_products['discount_price'] = df_products['discount_price'].str.replace('₹', '')
df_products['discount_price'] = df_products['discount_price'].str.replace(',', '')

# Karena bekerja dengan currency kita akan menggunaka datatype float
df_products['discount_price'] = df_products['discount_price'].astype(float)

In [35]:
# Bersihkan string agar dapat diubah
df_products['actual_price'] = df_products['actual_price'].str.replace('₹', '')
df_products['actual_price'] = df_products['actual_price'].str.replace(',', '')

# Karena bekerja dengan currency kita akan menggunaka datatype float
df_products['actual_price'] = df_products['actual_price'].astype(float)

In [36]:
# Kita juga membuat kolom baru untuk menyatukan harga yang sudah didiskon dan values yang tidak ada diskonnya.

def getCopyPrice(disc, actual):
    if pd.isnull(disc) and pd.notnull(actual):
        return actual
    else:
        return disc
    

df_products['current_price'] = df_products.apply(lambda x: getCopyPrice(x['discount_price'], x['actual_price']), axis=1)

In [37]:
df_products[df_products['discount_price'].isnull()].head()

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,Unnamed: 0,current_price
76,LG 1.5 Ton 3 Star Hot & Cold DUAL Inverter Spl...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51hbo8yQ1E...,https://www.amazon.in/LG-Inverter-Convertible-...,4.0,265,NaN,NaN,NaN,NaN
100,Hitachi 1.5 Ton 5 Star Inverter Split AC (Copp...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41AY1pk5oR...,https://www.amazon.in/Hitachi-Inverter-Copper-...,3.8,748,NaN,NaN,NaN,NaN
114,Panasonic 1.5 Ton 4 Star Wi-Fi Twin-Cool Inver...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41Edvsb7Gh...,https://www.amazon.in/Panasonic-Conditioner-An...,4.5,195,NaN,NaN,NaN,NaN
126,"Hitachi 1.5 Ton 5 Star Window AC (Copper, Dust...",appliances,Air Conditioners,https://m.media-amazon.com/images/I/81Ei0pUgd7...,https://www.amazon.in/Hitachi-Window-Copper-Fi...,2.7,10,NaN,49300.0,NaN,49300.0
137,"Portable Air Conditioner,Office Air Conditione...",appliances,Air Conditioners,https://m.media-amazon.com/images/I/61zHq-twHL...,https://www.amazon.in/Portable-Conditioner-Eva...,3.2,303,NaN,7250.0,NaN,7250.0


Karena kita sudah mentransformasi masing-masing kolom menjadi data type yang sesuai kita akan menyingkirkan kolom ```Unnamed: 0``` karena dinilai tidak relevan dengan kebutuh analisis data.

In [38]:
df_products = df_products.drop('Unnamed: 0', axis=1)

In [39]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103170 entries, 0 to 1103169
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   name            1103170 non-null  string 
 1   main_category   1103170 non-null  string 
 2   sub_category    1103170 non-null  string 
 3   image           1103170 non-null  string 
 4   link            1103170 non-null  string 
 5   ratings         739116 non-null   float64
 6   no_of_ratings   739053 non-null   Int64  
 7   discount_price  980844 non-null   float64
 8   actual_price    1067544 non-null  float64
 9   current_price   1067544 non-null  float64
dtypes: Int64(1), float64(4), string(5)
memory usage: 85.2 MB


## Data Demography

Dari kolom yang kita miliki, terdapat 2 kolom kategori yang bisa digunakan untuk melihat pembagian kelompok dengan aggregate functions yaitu `main_category` dan `sub_category`. Lalu kita memiliki kolom values yang bisa mendapatkan insight dari nilai `ratings`, `discount_price` yang akan kita olah untuk melihat besar diskon, dan `actual_price` untuk mendapatkan insight mengenai harga.

In [40]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103170 entries, 0 to 1103169
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   name            1103170 non-null  string 
 1   main_category   1103170 non-null  string 
 2   sub_category    1103170 non-null  string 
 3   image           1103170 non-null  string 
 4   link            1103170 non-null  string 
 5   ratings         739116 non-null   float64
 6   no_of_ratings   739053 non-null   Int64  
 7   discount_price  980844 non-null   float64
 8   actual_price    1067544 non-null  float64
 9   current_price   1067544 non-null  float64
dtypes: Int64(1), float64(4), string(5)
memory usage: 85.2 MB


In [41]:
# Melihat unique kolom 'main_category'

df_products['main_category'].unique()

<StringArray>
[             'appliances',    'toys & baby products',
             "men's shoes",          'bags & luggage',
         'car & motorbike',                  'stores',
          "men's clothing",          'home & kitchen',
        "women's clothing",             'accessories',
     'tv, audio & cameras',         'beauty & health',
        'sports & fitness',           "kids' fashion",
                   'music', 'grocery & gourmet foods',
            'pet supplies',           "women's shoes",
     'home, kitchen, pets',     'industrial supplies']
Length: 20, dtype: string

In [42]:
# Melihat unique kolom 'sub_category'

df_products['sub_category'].unique()

<StringArray>
[        'Air Conditioners',        'Nursing & Feeding',
             'Casual Shoes',                  'Wallets',
          'Car Accessories',                  'Diapers',
           'Amazon Fashion',         'T-shirts & Polos',
            'Bedroom Linen',             'Toys & Games',
 ...
             'Sports Shoes',          'STEM Toys Store',
        'Strollers & Prams', 'Suitcases & Trolley Bags',
              'Televisions',  'Test, Measure & Inspect',
    'The Designer Boutique',             'Value Bazaar',
             'Western Wear',                     'Yoga']
Length: 112, dtype: string

In [43]:
# Untuk melihat secara general, dataframe berbentuk seperti apa

df_products.describe()

,ratings,no_of_ratings,discount_price,actual_price,current_price
count,739116.000000,739053.0,9.808440e+05,1.067544e+06,1.067544e+06
mean,3.832311,840.847039,2.623161e+03,2.311128e+04,2.872724e+03
std,0.756101,8651.895695,9.458191e+03,1.355081e+07,9.565771e+03
min,1.000000,1.0,8.000000e+00,0.000000e+00,0.000000e+00
25%,3.500000,4.0,3.890000e+02,9.900000e+02,3.990000e+02
50%,3.900000,20.0,6.790000e+02,1.599000e+03,6.990000e+02
75%,4.300000,133.0,1.399000e+03,2.999000e+03,1.549000e+03
max,5.000000,589547.0,1.249990e+06,9.900000e+09,1.249990e+06


### Analisis ratings

In [44]:
# Mengagregat value ratings dengan pembagian kelompok kolom 'main_category'

df_products.groupby('main_category')['ratings'].agg(['mean', 'min', 'max', 'count']).sort_values(by='mean', ascending=False)

,mean,min,max,count
main_category,,,,
grocery & gourmet foods,4.072739,1.0,5.0,6038
toys & baby products,4.030417,1.0,5.0,10514
pet supplies,4.029325,1.0,5.0,2844
beauty & health,4.002085,1.0,5.0,15446
home & kitchen,3.977118,1.0,5.0,26554
music,3.964741,1.0,5.0,1696
stores,3.946108,1.0,5.0,56728
bags & luggage,3.939759,1.0,5.0,7314
accessories,3.904224,1.0,5.0,134838


In [45]:
# Mengagregat value ratings dengan pembagian kelompok kolom 'main_category' & sub category

df_products.groupby(['main_category', 'sub_category'])['ratings'].agg(['mean', 'min', 'max', 'count']).sort_values(by='mean', ascending=False)

,,mean,min,max,count
main_category,sub_category,,,,
toys & baby products,International Toy Store,4.578947,4.0,4.9,38
beauty & health,Value Bazaar,4.304545,3.7,4.6,132
grocery & gourmet foods,All Grocery & Gourmet Foods,4.176296,2.0,5.0,1890
beauty & health,Health & Personal Care,4.174317,2.4,5.0,2196
toys & baby products,Strollers & Prams,4.127092,1.0,5.0,502
...,...,...,...,...,...
"tv, audio & cameras",Headphones,3.503177,1.0,5.0,13786
car & motorbike,Car Electronics,3.497718,1.0,5.0,1490
beauty & health,Personal Care Appliances,3.455056,1.0,5.0,712


Dengan dua analisis diatas, kita menemukan bahwa 'grocery & gourmet foods' dalam `main_category` memiliki rata2 `rating` terbesar diikuti oleh dengan 'toys & baby products' dan 'beauty & health'.

Melihat lebih detail ke `sub_category`, kita menemukan bahwa hasil tetap konsisten dengan `main_category` sebelumnya.  Dipimpin 'international toy store', 'value bazaar;, dan 'all grocery & gourmet foods' dalam `sub_category`. 

kedua `main_category` dan `sub_category` konsisten dengan urutan terakhir `main_category` dengan 'home, kitchen, pets 'Refurbished & Open Box'.

### Prices

In [46]:
# Mengagregat value current_price dengan pembagian kelompok kolom 'main_category'.

df_products.groupby('main_category')['current_price'].agg(['mean', 'min', 'max', 'count']).sort_values(by='mean', ascending=False)

,mean,min,max,count
main_category,,,,
accessories,5615.944755,4.00,499999.00,222342
appliances,5463.668770,20.00,230000.00,62652
music,4004.035453,50.00,76900.00,2076
stores,3922.920501,0.00,406009.00,64670
"home, kitchen, pets",3643.705882,1199.00,8541.00,34
"tv, audio & cameras",3427.445350,0.00,1249990.00,132714
men's shoes,2600.654225,25.00,66649.00,109726
bags & luggage,2317.375915,10.00,140000.00,19964
industrial supplies,2309.831312,35.94,144856.00,8020


In [47]:
# Mengagregat value 'current_prices' dengan menambahkan pembagian kolom 'sub_category'

df_products.groupby(['main_category', 'sub_category'])['current_price'].agg(['mean', 'min', 'max', 'count']).sort_values(by='mean', ascending=False)

,,mean,min,max,count
main_category,sub_category,,,,
"tv, audio & cameras",Televisions,39849.728827,100.0,1249990.0,1466
appliances,Air Conditioners,38725.549240,199.0,128800.0,1000
accessories,Gold & Diamond Jewellery,29929.124631,110.0,468012.0,31486
sports & fitness,Cardio Equipment,28846.721122,235.0,225250.0,392
appliances,Refrigerators,19792.077584,99.0,189200.0,3278
...,...,...,...,...,...
grocery & gourmet foods,"Coffee, Tea & Beverages",498.725428,8.0,7900.0,2568
beauty & health,Beauty & Grooming,481.695149,19.0,9200.0,3830
home & kitchen,Sewing & Craft Supplies,438.800707,35.0,15999.0,2518


Dengan dua analisis diatas, kita dapat lihat bahwa walaupun 'appliences' dan 'accesories' merupakan `main_category` dengan nilai rata-rata `current_price` paling tinggi. Cukup mengejutkan dalam `sub_category` dipuncaki oleh 'televisions' dimana `main_category`nya merupakan urutan ketujuh dalam `main_category`.

Hal ini bisa kita lihat karena memang 'televisions' memiliki price max yang sangat besar dengan 1249990. 

### Discount

In [48]:
# Membuat kolom discount

df_products['discount'] = df_products['actual_price'] - df_products['current_price']

In [49]:
df_products.groupby('main_category')['discount'].agg(['mean', 'max']).sort_values(by='mean', ascending=False)

,mean,max
main_category,,
home & kitchen,684923.029268,9.899999e+09
sports & fitness,6333.442432,6.108230e+07
appliances,2668.417563,3.700000e+05
accessories,2520.361441,7.939910e+05
"tv, audio & cameras",2298.101095,3.449100e+05
bags & luggage,1730.531382,9.600000e+04
music,1617.782216,5.244100e+04
men's shoes,1252.831183,2.695600e+04
industrial supplies,1136.832185,4.505000e+04


In [50]:
df_products.groupby(['main_category', 'sub_category'])['discount'].agg(['mean', 'sum', 'count']).sort_values(by='mean', ascending=False)

,,mean,sum,count
main_category,sub_category,,,
home & kitchen,Garden & Outdoors,9.017007e+06,1.980135e+10,2196
sports & fitness,Running,6.834569e+04,1.237057e+08,1810
"tv, audio & cameras",Televisions,2.238134e+04,3.281104e+07,1466
sports & fitness,Cardio Equipment,1.857530e+04,7.281519e+06,392
appliances,Air Conditioners,1.608968e+04,1.608968e+07,1000
...,...,...,...,...
beauty & health,Health & Personal Care,2.381061e+02,5.228810e+05,2196
grocery & gourmet foods,Snack Foods,1.970310e+02,4.114006e+05,2088
beauty & health,Value Bazaar,1.579241e+02,2.084598e+04,132


Hasil yang ditemuka dari analisis diatas konsisten untuk jumlah rata-rata diskon diuruti paling tinggi dalam `main_category` adalah 'home & kitchen' dan 'sport & fitness' dan juga konsisten dalam `sub_category` yang dipimpin oleh 'Garden & Outdoors' dan 'Running.

'televisions' muncul kembali dalam `sub_category` urutan ketiga.

Kita juga akan melihat bagaimana tidak hanya diskon, tetapi juga persentase diskon dalam suatu produk.

In [51]:
# Membuat kolom 'discount percentage'

df_products['discount_percentage'] = (df_products['actual_price'] - df_products['current_price'])/df_products['actual_price']

In [52]:
df_products.groupby('main_category')['discount_percentage'].agg(['mean', 'max']).sort_values(by='mean', ascending=False)

,mean,max
main_category,,
women's clothing,0.565146,0.949703
accessories,0.505547,0.999000
"tv, audio & cameras",0.470715,0.991559
home & kitchen,0.458333,1.000000
bags & luggage,0.457284,0.999000
sports & fitness,0.445736,0.999989
car & motorbike,0.433778,0.979098
kids' fashion,0.422090,0.959111
men's clothing,0.412937,0.945541


In [53]:
df_products.groupby(['main_category', 'sub_category'])['discount_percentage'].agg(['mean', 'sum']).sort_values(by='mean', ascending=False)

mean           sum
main_category           sub_category                                       
accessories             Fashion & Silver Jewellery   0.643045  24289.093307
women's clothing        Ethnic Wear                  0.627406  23551.569500
                        Clothing                     0.595726  22676.908288
accessories             Jewellery                    0.575426  21667.077817
women's clothing        Western Wear                 0.558816  21251.765064
...                                                       ...           ...
grocery & gourmet foods All Grocery & Gourmet Foods  0.176166    336.124467
home, kitchen, pets     Refurbished & Open Box       0.172337      5.859442
stores                  Sportswear                   0.148848   2118.100738
grocery & gourmet foods Coffee, Tea & Beverages      0.147204    378.018619
toys & baby products    International Toy Store      0.121031      5.567447

[112 rows x 2 columns]

Hasil yang ditemukan cukup mengejutkan dikarenakan hasil analisis `discount_percentage` dan `discount` cukup berbeda.

Perbedaan itu dapat dilihat dengan bagaiaman `main_category` dipimpin oleh 'women's clothing', 'accesories', dan 'tv, audio, & cameras' dalam sisi `discount_percentage`. Hal ini konsisten dengan analisis `sub_category` dipimpin dengan sub-kategori yang memiliki `main_category` yang konsisten dengan analisis. Dipimpin oleh 'Fashing & Silver Jewellery', 'Ethnic Wear', & 'Clothing'.